# 13. Modeling (Stata) - all commodities

In [1]:
use "../stata_data/low_change_sample_features_2019-09-07.dta"

In [2]:
ds

YEAR          MILES         MILES_1950    MILES_1990    INCIDENTS     AVG_AGE       change_2      miles_avg_3   age_miles     perc_1960     perc_2000     avg_1950      avg_1990      SIGN_INC_M~E  M_A_past_3    sd_change_sq
COMMODITY     AGE_UNKNOW~S  MILES_1960    MILES_2000    SIGNIFICAN~S  CHANGE        change_3      miles_1950_3  perc_pr~1940  perc_1970     perc_2010     avg_1960      avg_2000      M_A_1         LOG_INCIDE~E  sd_chang~w_3
OPERATOR_ID   MILES_P~1940  MILES_1970    MILES_2010    GROUP         M_A           mean_chang~3  sign_inc_3    perc_1940     perc_1980     avg_pre_1940  avg_1970      avg_2010      M_A_2         LOG_SIGN_I~E  one_over_s~e
NAME          MILES_1940    MILES_1980    PARENT        PERC_OFFSH~E  change_1      sd_change_3   inc_per_mi~3  perc_1950     perc_1990     avg_1940      avg_1980      INCIDENTS_~E  M_A_3         log_sd_cha~e  miles_x_ch~e


In [3]:
egen panelid = group(OPERATOR_ID COMMODITY), label

In [4]:
xtset panelid YEAR

       panel variable:  panelid (unbalanced)
        time variable:  YEAR, 2009 to 2018, but with gaps
                delta:  1 unit


In [16]:
xtreg inc_per_mile_3 sd_change_3 avg_1940 avg_1950 avg_1960 avg_1970 avg_1980 avg_1990 avg_2000 avg_2010, fe vce(robust)


Fixed-effects (within) regression               Number of obs     =        440
Group variable: panelid                         Number of groups  =        102

R-sq:                                           Obs per group:
     within  = 0.0280                                         min =          1
     between = 0.0001                                         avg =        4.3
     overall = 0.0006                                         max =          5

                                                F(9,101)          =       1.65
corr(u_i, Xb)  = -0.5453                        Prob > F          =     0.1121

                              (Std. Err. adjusted for 102 clusters in panelid)
------------------------------------------------------------------------------
             |               Robust
inc_per_mi~3 |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
 sd_change_3 |  -6.48e-06   

In [6]:
local date : di  %tdCY-N-D  daily("$S_DATE", "DMY")

In [7]:
putexcel set "../results/fe_small_`date'.xlsx", replace

Note: file will be replaced when the first putexcel command is issued


In [8]:
matrix D = r(table)'

In [9]:
putexcel A1 = matrix(D), names

file ../results/fe_small_2019-09-07.xlsx saved


## Hausman test

In [10]:
xtreg inc_per_mile_3 sd_change_3 M_A M_A_1 M_A_2 avg_1940 avg_1950 avg_1960 avg_1970 avg_1980 avg_1990 avg_2000 avg_2010, fe

note: M_A omitted because of collinearity

Fixed-effects (within) regression               Number of obs     =        440
Group variable: panelid                         Number of groups  =        102

R-sq:                                           Obs per group:
     within  = 0.0287                                         min =          1
     between = 0.0001                                         avg =        4.3
     overall = 0.0007                                         max =          5

                                                F(11,327)         =       0.88
corr(u_i, Xb)  = -0.5406                        Prob > F          =     0.5608

------------------------------------------------------------------------------
inc_per_mi~3 |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
 sd_change_3 |  -5.10e-06   7.62e-06    -0.67   0.504    -.0000201    9.89e-06
         M_A |        

In [11]:
estimate store fixed

In [12]:
xtreg inc_per_mile_3 sd_change_3 M_A M_A_1 M_A_2 avg_1940 avg_1950 avg_1960 avg_1970 avg_1980 avg_1990 avg_2000 avg_2010, re

note: M_A omitted because of collinearity

Random-effects GLS regression                   Number of obs     =        440
Group variable: panelid                         Number of groups  =        102

R-sq:                                           Obs per group:
     within  = 0.0111                                         min =          1
     between = 0.0615                                         avg =        4.3
     overall = 0.0457                                         max =          5

                                                Wald chi2(11)     =       9.89
corr(u_i, X)   = 0 (assumed)                    Prob > chi2       =     0.5400

------------------------------------------------------------------------------
inc_per_mi~3 |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
 sd_change_3 |  -3.84e-06   6.41e-06    -0.60   0.549    -.0000164    8.72e-06
         M_A |        

In [13]:
hausman fixed ., sigmamore


Note: the rank of the differenced variance matrix (10) does not equal the number of coefficients being tested (11); be sure this is what you expect, or there may be problems computing the test.  Examine the output of your estimators for anything
        unexpected and possibly consider scaling your variables so that the coefficients are on a similar scale.

                 ---- Coefficients ----
             |      (b)          (B)            (b-B)     sqrt(diag(V_b-V_B))
             |     fixed          .          Difference          S.E.
-------------+----------------------------------------------------------------
 sd_change_3 |   -5.10e-06    -3.84e-06       -1.26e-06        4.14e-06
       M_A_1 |    .0011374     .0043856       -.0032482         .005485
       M_A_2 |   -.0022222     .0005347       -.0027569        .0050056
    avg_1940 |    .0089595     .0029904         .005969        .0143796
    avg_1950 |    .0008115     .0109837       -.0101723        .0149691
    avg_1960